In [1]:
import praw
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import pandas as pd
import datetime as dt
import re 

## API

In [2]:
from pmaw import PushshiftAPI
api = PushshiftAPI()

## select subreddit

In [3]:
subreddit="politics"
limit=1000

# adding timeframe 
before = int(dt.datetime(2022,1,1,1,0).timestamp())
after = int(dt.datetime(2022,1,1,0,0).timestamp())

comments = api.search_comments(subreddit=subreddit, limit=limit, before=before, after=after)
print(f'Retrieved {len(comments)} comments from Pushshift')

Retrieved 1000 comments from Pushshift


## Start scraping
Scraping the above selected subreddit for every day. Then saving it in a csv for per year.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
temp_df = pd.DataFrame() #Temporary empty dataframe

for m in range(1, 13):
    year = 2020
    yearAfter = year
    monthBefore = m
    monthAfter = m
    day = 32
    minutes = 0
    
    # YEARLY SCRAPE
    # If year has comments < 1000 scrape everything at once 
    scrapeYearDateBefore = int(dt.datetime(year,12,31,0,0).timestamp())
    scrapeYearDateAfter = int(dt.datetime(year,1,1,0,0).timestamp())
       
    # Scrape all comments per Year
    commentsYear = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeYearDateBefore, after=scrapeYearDateAfter)
    
    
    if (len(commentsYear) < 1000): 
        print(f'Retrieved {len(commentsYear)} comments in {year} from Pushshift')
        # convert comments into a datafrom
        comments_df = pd.DataFrame(commentsYear)
        
        # Filter on keywords Climate and Change
        for index, row in comments_df.iterrows():
            text = row['body']
            regex_climate = re.compile(r"\b[Cc]limate")
            regex_change = re.compile(r"\b[Cc]hange")
            if regex_climate.search(text) and regex_change.search(text):
                # if the string contains both 'Climate' and 'Change',
                # add the row to the new dataframe
                temp_df = temp_df.append(row, ignore_index=True)
    

    
    
    # MONTLY SCRAPE
    else:
        print(f'more than {len(commentsYear)} comments in year {year}')
        endMonthDate = 31
        
        if(m == 1) or (m == 3) or (m == 5) or (m == 7) or (m == 8) or (m == 10) or (m == 12):
            endMonthDate = 31
        elif(m == 2):
            endMonthDate = 28
        else: 
            endMonthDate = 30
            
        # If month has comments < 1000 scrape all monthly data at once 
        scrapeMonthDateBefore = int(dt.datetime(year,m,endMonthDate,0,0).timestamp())
        scrapeMonthDateAfter = int(dt.datetime(year,m,1,0,0).timestamp())
        
        # Scrape all comments per day
        commentsMonth = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeMonthDateBefore, after=scrapeMonthDateAfter)
        
        if (len(commentsMonth) < 1000): 
            print(f'Retrieved {len(commentsMonth)} comments in month: {m} - from Pushshift')
            # convert comments into a datafrom
            comments_df = pd.DataFrame(commentsMonth)

            # Filter on keywords Climate and Change
            for index, row in comments_df.iterrows():
                text = row['body']
                regex_climate = re.compile(r"\b[Cc]limate")
                regex_change = re.compile(r"\b[Cc]hange")
                if regex_climate.search(text) and regex_change.search(text):
                    # if the string contains both 'Climate' and 'Change',
                    # add the row to the new dataframe
                    temp_df = temp_df.append(row, ignore_index=True)

    
    
    
        # DAILY SCRAPE
        else:
            print(f'more than {len(commentsMonth)} comments in month {m} - {year}')
            if (m == 2): #februari 
                day = 29
            if (m == 4) or (m == 6) or (m == 9) or (m == 11): # months with 30 days -> its days +1 for the loop
                day = 31

            for d in range(1, day):
                # date variables
                dayBefore = d
                dayAfter = d-1

                # change dayAfter / monthAfter on certain conditions
                if(dayAfter == 0): # 
                    monthAfter = m-1
                    if(m == 2):
                        dayAfter = 31
                    elif(m==3):
                        dayAfter = 28
                    elif (m == 4) or (m == 6) or (m == 9) or (m == 11) or (m == 13):
                        dayAfter = 31
                    else: dayAfter = 30
                else: monthAfter = m

                if(monthAfter == 0):
                    continue

                #set time frame for scraping -> Scraping for every day in the year.
                scrapeDayDateBefore = int(dt.datetime(year,monthBefore,dayBefore,0,0).timestamp())
                scrapeDayDateAfter = int(dt.datetime(yearAfter,monthAfter,dayAfter,0,0).timestamp())


                # Scrape all comments per day
                commentsDay = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeDayDateBefore, after=scrapeDayDateAfter)
                  
                # if statement day comments < 1000
                if (len(commentsDay) < 1000): 
                    print('datebefore', dayBefore, '-', monthBefore, '- ', year, 'date after', dayAfter, '-', monthAfter, '- ', yearAfter)
                    print(f'Retrieved {len(commentsDay)} comments per day from Pushshift')
                    # convert comments into a datafrom
                    comments_df = pd.DataFrame(commentsDay)

                    # Filter on keywords Climate and Change
                    for index, row in comments_df.iterrows():
                        text = row['body']
                        regex_climate = re.compile(r"\b[Cc]limate")
                        regex_change = re.compile(r"\b[Cc]hange")
                        if regex_climate.search(text) and regex_change.search(text):
                            # if the string contains both 'Climate' and 'Change',
                            # add the row to the new dataframe
                            temp_df = temp_df.append(row, ignore_index=True)
      
                # 12 HOURLY SCRAPE
                else:
                    print(f'more than {len(commentsDay)} comments on day {dayAfter}-{monthAfter}-{yearAfter} -- {dayBefore}-{monthBefore}-{year}')
                    print(f'attempting 12-hourly scrape...')
                    for b in range(2):

                        hourMeasureBefore = 12
                        hourMeasureAfter = 0
                        minuteMeasureBefore = 0
                        
                        if (b == 0):
                            hourMeasureBefore = 12
                            hourMeasureAfter = 0
                            minuteMeasureBefore = 0

                        if (b == 1):
                            hourMeasureBefore = 23
                            hourMeasureAfter = 12
                            minuteMeasureBefore = 0

                        
                        scrapeTwelveHourDateBefore = int(dt.datetime(year,m,d,hourMeasureBefore,minuteMeasureBefore).timestamp())
                        scrapeTwelveHourDateAfter = int(dt.datetime(year,m,d,hourMeasureAfter,0).timestamp())  


                        # Scrape all comments per hour
                        commentsTwelveHour = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeTwelveHourDateBefore, after=scrapeTwelveHourDateAfter)

                        # if statement hour comments < 1000
                        if (len(commentsTwelveHour) < 1000): 
                            print('datebefore', dt.datetime(year,m,d,hourMeasureBefore,minuteMeasureBefore), 'date after', dt.datetime(yearAfter,m,d,hourMeasureAfter,0)) 
                            print(f'Retrieved {len(commentsTwelveHour)} comments per 12 hours from Pushshift')
                            # convert comments into a datafrom
                            comments_df = pd.DataFrame(commentsTwelveHour)

                            # Filter on keywords Climate and Change
                            for index, row in comments_df.iterrows():
                                text = row['body']
                                regex_climate = re.compile(r"\b[Cc]limate")
                                regex_change = re.compile(r"\b[Cc]hange")
                                if regex_climate.search(text) and regex_change.search(text):
                                    # if the string contains both 'Climate' and 'Change',
                                    # add the row to the new dataframe
                                    temp_df = temp_df.append(row, ignore_index=True)

    
    
    
                        # HOURLY SCRAPE
                        else: 
                            print(f'more than {len(commentsTwelveHour)} comments: failed 12-hourly scrape, attempting hourly scrape...')
                            
                            for h in range(24):

                                # add an hour to the datetime object
                                hourBefore = h + 1
                                hourAfter = h
                                minuteAfter = 0

                                if (hourBefore == 24):
                                    hourBefore = 23
                                    minutes = 59
                                else: 
                                    hourBefore = h + 1
                                    minutes = 0



                                #set time frame for scraping -> Scraping for every hour in the day
                                scrapeHourDateBefore = int(dt.datetime(year,m,d,hourBefore,minutes).timestamp())
                                scrapeHourDateAfter = int(dt.datetime(yearAfter,m,d,hourAfter,minuteAfter).timestamp())      

                                # Scrape all comments per hour
                                commentsHour = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeHourDateBefore, after=scrapeHourDateAfter)

                                # if statement hour comments < 1000
                                if (len(commentsHour) < 1000): 
                                    print('datebefore', dt.datetime(year,m,d,hourBefore,minutes), 'date after', dt.datetime(yearAfter,m,d,hourAfter,minuteAfter)) 
                                    print(f'Retrieved {len(commentsHour)} comments per hour from Pushshift')
                                    # convert comments into a datafrom
                                    comments_df = pd.DataFrame(commentsHour)

                                    # Filter on keywords Climate and Change
                                    for index, row in comments_df.iterrows():
                                        text = row['body']
                                        regex_climate = re.compile(r"\b[Cc]limate")
                                        regex_change = re.compile(r"\b[Cc]hange")
                                        if regex_climate.search(text) and regex_change.search(text):
                                            # if the string contains both 'Climate' and 'Change',
                                            # add the row to the new dataframe
                                            temp_df = temp_df.append(row, ignore_index=True)

                                            
                                            
                                # SCRAPING PER 20 MINUTES
                                else:
                                    print(f'more than {len(commentsHour)} per hour: failed to do an hourly scrape, attempting 20 minute scrapes scrape...')
                                
                                    for q in range(3):

                                        minuteBefore = 20
                                        minuteAfter = 0

                                        if (q == 0):
                                            minuteBefore = 20
                                            minuteAfter = 0

                                        if (q == 1):
                                            minuteBefore = 40
                                            minuteAfter = 20
                                            
                                        if (q == 2):
                                            minuteBefore = 59
                                            minuteAfter = 40


                                        scrapeMinuteDateBefore = int(dt.datetime(year,m,d,h,minuteBefore).timestamp())
                                        scrapeMinuteDateAfter = int(dt.datetime(year,m,d,h,minuteAfter).timestamp())  


                                        # Scrape all comments per hour
                                        commentsMinute = api.search_comments(subreddit=subreddit, limit=limit, before=scrapeMinuteDateBefore, after=scrapeMinuteDateAfter)

                                        # if statement hour comments < 1000
                                        if (len(commentsMinute) < 1000): 
                                            print('dateafter', dt.datetime(year,m,d,h,minuteAfter), 'date after', dt.datetime(year,m,d,h,minutesBefore)) 
                                            print(f'Retrieved {len(commentsMinute)} comments per 20 minutes from Pushshift')
                                            # convert comments into a datafrom
                                            comments_df = pd.DataFrame(commentsMinute)

                                            # Filter on keywords Climate and Change
                                            for index, row in comments_df.iterrows():
                                                text = row['body']
                                                regex_climate = re.compile(r"\b[Cc]limate")
                                                regex_change = re.compile(r"\b[Cc]hange")
                                                if regex_climate.search(text) and regex_change.search(text):
                                                    # if the string contains both 'Climate' and 'Change',
                                                    # add the row to the new dataframe
                                                    temp_df = temp_df.append(row, ignore_index=True)                                    
                                    
                                    
                                    
                                        else:
                                        # convert comments into a datafrom
                                            comments_df = pd.DataFrame(commentsMinute)

                                            # Filter on keywords Climate and Change
                                            for index, row in comments_df.iterrows():
                                                text = row['body']
                                                regex_climate = re.compile(r"\b[Cc]limate")
                                                regex_change = re.compile(r"\b[Cc]hange")
                                                if regex_climate.search(text) and regex_change.search(text):
                                                    # if the string contains both 'Climate' and 'Change',
                                                    # add the row to the new dataframe
                                                    temp_df = temp_df.append(row, ignore_index=True)

                                            print('ERROR MORE THAN 1000 per 20 minutes, however, still going on. INCOMPLETE')
                                            print(f'more than {len(commentsMinute)} comments on day {d}-{m}-{year} between {h}:{minutesAfter} and {h}:{minutesBefore}')
                #                             raise ValueError("More than 1000 comments in this HOUR.")

                                
                                
                                
                                
                                

# all datapoints in temporary dataframe to one dataframe
df1 = pd.DataFrame(temp_df)
                                      
# create the filename with the variable name embedded
filename = f'{subreddit}_{year}.csv'

# save the dataframe to the file with the embedded variable name
df1.to_csv(filename, encoding='utf-8', header=True, index=False)


more than 1000 comments in year 2020
more than 1000 comments in month 1 - 2020
more than 1000 comments on day 1-1-2020 -- 2-1-2020
attempting 12-hourly scrape...
failed 12-hourly scrape, attempting hourly scrape...
more than 1000 comments on day 1-1-2020 -- 2-1-2020
failed to do an hourly scrape, attempting 20 minute scrapes scrape...
more than 1000 comments in hours 2-1-2020 0:0 -- 1:0
datebefore 2020-01-02 12:00:00 date after 2020-01-02 00:00:00
Retrieved 994 comments per 12 hours from Pushshift
datebefore 2020-01-02 12:00:00 date after 2020-01-02 00:00:00
Retrieved 857 comments per 12 hours from Pushshift
datebefore 2020-01-02 12:00:00 date after 2020-01-02 00:00:00
Retrieved 768 comments per 12 hours from Pushshift
failed to do an hourly scrape, attempting 20 minute scrapes scrape...
more than 1000 comments in hours 2-1-2020 1:0 -- 2:0
datebefore 2020-01-02 12:00:00 date after 2020-01-02 00:00:00
Retrieved 766 comments per 12 hours from Pushshift
datebefore 2020-01-02 12:00:00 date

In [ ]:
print(len(df1))